In [1]:
import os
import cv2
import numpy as np
from keras.models import model_from_json

In [2]:
label_dicts ={0:"ANGRY",1:"DISGUSTED",2:"FEAR",3:"HAPPY",4:"NEUTRAL",5:"SAD",6:"SURPRISED"}

In [3]:
json_file=open('emotion_model.json','r')
loaded_model_json=json_file.read()
json_file.close()
emotion_model=model_from_json(loaded_model_json)

In [4]:
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
while True:
    # Find haar cascade to draw bounding box around face
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier("C:/Users/maanu/AppData/Local/Programs/Python/Python38/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#"C:\Users\maanu\AppData\Local\Programs\Python\Python38\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml"
    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, label_dicts[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    key = cv2.waitKey(1)
    if key == 27: 
        break


cap.release()
cv2.destroyAllWindows()